# Morris散点图

## fei+peo+seeo

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import os
import numpy as np
from adjustText import adjust_text

# 全局字体
matplotlib.rcParams['font.family'] = 'Times New Roman'
matplotlib.rcParams['font.sans-serif'] = ['SimSun', 'Times New Roman']
matplotlib.rcParams['axes.unicode_minus'] = False

# 文件路径和标签
csv_files = [
    (r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\peo\1_综合比较不同敏感度指标\peo_SHAP_vs_Imp.csv", "a"),
    (r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\fei\1_综合比较不同敏感度指标\fei_SHAP_vs_Imp.csv", "b"),
    (r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\seeo\1_综合比较不同敏感度指标\seeo_SHAP_vs_Imp.csv", "c"),
]
outdir = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\图像优化"
os.makedirs(outdir, exist_ok=True)
outpath = os.path.join(outdir, "XGBoost变量归因_Morris优化.png")

# 智能文本调整
def smart_adjust_text(texts, ax, max_shrinkA=27, start_shrinkA=7, step=5):
    for shrinkA in range(start_shrinkA, max_shrinkA+1, step):
        try:
            adjust_text(
                texts, ax=ax,
                expand_points=(2.8, 3.1),
                arrowprops=dict(arrowstyle="-", color="black", lw=1.25, alpha=0.82, shrinkA=shrinkA)
            )
            break
        except Exception:
            continue

# 绘制单个子图
def plot_morris(df, ax, mono_cmap, combo_exp=3.2):
    def norm(x): return (x - x.min()) / (x.max() - x.min() + 1e-8)
    norm_SHAP = norm(df["均值_SHAP"])
    norm_STD = norm(df["SHAP_标准差"])
    norm_imp = norm(df["特征重要性"])
    combo = 0.4 * norm_SHAP + 0.4 * norm_STD + 0.2 * norm_imp
    size = 180 + 9200 * (combo ** combo_exp)
    bubble_colors = mono_cmap(norm_imp)

    # 绘制气泡
    for i in range(len(df)):
        ax.scatter(
            df["均值_SHAP"][i], df["SHAP_标准差"][i],
            s=size[i], color=bubble_colors[i],
            alpha=0.77, linewidths=0, zorder=2
        )

    # 添加标签
    texts = []
    for i in range(len(df)):
        if size[i] > 2200:  # 大气泡 → 居中
            texts.append(ax.text(
                df["均值_SHAP"][i], df["SHAP_标准差"][i],
                str(df["变量"][i]),
                fontsize=18 + int(9 * combo[i]),
                color='black',
                ha='center', va='center', zorder=10,
                fontweight='bold', fontfamily='Times New Roman',
                bbox=dict(boxstyle="round,pad=0.18", fc="white", alpha=0.82, lw=0)
            ))
        else:  # 小气泡 → 相对偏移
            dx = 0.02 * (df["均值_SHAP"].max() - df["均值_SHAP"].min())
            dy = 0.02 * (df["SHAP_标准差"].max() - df["SHAP_标准差"].min())

            texts.append(ax.text(
                df["均值_SHAP"][i] + dx, df["SHAP_标准差"][i] + dy,
                str(df["变量"][i]),
                fontsize=18 + int(6 * combo[i]),
                color='black',
                ha='left', va='bottom', zorder=10,
                fontweight='bold', fontfamily='Times New Roman',
                bbox=dict(boxstyle="round,pad=0.13", fc="white", alpha=0.75, lw=0)
            ))

    smart_adjust_text(texts, ax)

    # 坐标轴 & 样式
    ax.set_ylabel("Morris σ", fontsize=30, fontweight='bold',
                  color='black', labelpad=15, fontfamily='Times New Roman')
    ax.set_xlabel("Morris µ*", fontsize=30, fontweight='bold',
                  color='black', fontfamily='Times New Roman', labelpad=11)
    ax.grid(True, linestyle="--", alpha=0.13, zorder=0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.tick_params(axis='y', labelsize=30)
    ax.tick_params(axis='x', labelsize=30)

    # 刻度字体不加粗
    for label in ax.get_yticklabels():
        label.set_fontname('Times New Roman')
    for label in ax.get_xticklabels():
        label.set_fontname('Times New Roman')

# 红色系 colormap
mono_cmap = plt.cm.Reds

# 2×2 布局（3张图 + 1空白隐藏）
fig, axes = plt.subplots(2, 2, figsize=(16, 16), sharex=False, constrained_layout=False)
axes = axes.flatten()

for i, (csv_path, label) in enumerate(csv_files):
    df = pd.read_csv(csv_path, encoding="utf-8-sig")
    ax = axes[i]
    plot_morris(df, ax, mono_cmap)

    # 子图角标
    ax.text(0.01, 0.97, label, transform=ax.transAxes,
            fontsize=30, fontweight='bold', va='top', ha='left',
            color='black', fontfamily='Times New Roman')

# 隐藏第4个子图
axes[-1].axis("off")

# 色条
all_min = min(pd.read_csv(f[0], encoding="utf-8-sig")["特征重要性"].min() for f in csv_files)
all_max = max(pd.read_csv(f[0], encoding="utf-8-sig")["特征重要性"].max() for f in csv_files)
norm = plt.Normalize(vmin=all_min, vmax=all_max)
sm = plt.cm.ScalarMappable(cmap=mono_cmap, norm=norm)
sm.set_array([])

cbar = fig.colorbar(
    sm,
    ax=axes.ravel().tolist(),
    orientation='vertical',
    shrink=0.80,
    pad=0.06,
    fraction=0.037,
    aspect=22
)
cbar.set_label(
    "特征重要性",
    fontsize=30, fontweight='bold', fontfamily='SimSun', color='black', labelpad=28
)
for label in cbar.ax.get_yticklabels():
    label.set_fontname('Times New Roman')
    label.set_fontsize(30)

cbar.ax.tick_params(colors='black', labelsize=15)
for spine in cbar.ax.spines.values():
    spine.set_edgecolor('black')

# 布局优化 & 保存
plt.subplots_adjust(left=0.09, right=0.86, top=0.96, bottom=0.07,
                    hspace=0.25, wspace=0.22)
plt.savefig(outpath, dpi=340, transparent=True)
plt.close()

print(f"[✔] 2×2 Morris 气泡图（相对偏移修正版）已保存至：{outpath}")


[✔] 2×2 Morris 气泡图（相对偏移修正版）已保存至：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\图像优化\XGBoost变量归因_Morris优化.png


## 去掉PEO

### 加粗去中文

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import os
import numpy as np
from adjustText import adjust_text

# --- 修改：全局字体设置，移除SimSun，确保默认使用Times New Roman ---
matplotlib.rcParams['font.family'] = 'Times New Roman'
# matplotlib.rcParams['font.sans-serif'] = ['SimSun', 'Times New Roman'] # 移除SimSun
matplotlib.rcParams['axes.unicode_minus'] = False

# 文件路径和标签 (无变化)
csv_files = [
    (r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\fei\1_综合比较不同敏感度指标\fei_SHAP_vs_Imp.csv", "a"),
    (r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\seeo\1_综合比较不同敏感度指标\seeo_SHAP_vs_Imp.csv", "b"),
]

# 输出文件夹 (无变化)
outdir = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\图像优化\去除PEO_剩FEI_SEEO"
os.makedirs(outdir, exist_ok=True)
outpath = os.path.join(outdir, "XGBoost_Morris优化_去PEO.png")


# 智能文本调整 (无变化)
# --- 建议修改后的函数定义 ---
def smart_adjust_text(texts, ax, max_shrinkA=35, start_shrinkA=15, step=5): # 进一步提高 shrinkA 的范围
    for shrinkA in range(start_shrinkA, max_shrinkA+1, step):
        try:
            adjust_text(
                texts, ax=ax,
                expand_points=(2.8, 3.1),
                arrowprops=dict(arrowstyle="-", color="black", lw=1.25, alpha=0.82, shrinkA=shrinkA)
            )
            break # 如果成功，就跳出循环
        except Exception:
            continue # 如果失败，继续尝试下一个值

# 绘制单个子图 (修改内部字体设置)
def plot_morris(df, ax, mono_cmap, combo_exp=3.2):
    def norm(x): return (x - x.min()) / (x.max() - x.min() + 1e-8)
    norm_SHAP = norm(df["均值_SHAP"])
    norm_STD = norm(df["SHAP_标准差"])
    norm_imp = norm(df["特征重要性"])
    combo = 0.4 * norm_SHAP + 0.4 * norm_STD + 0.2 * norm_imp
    
    size = 400 + 25000 * (combo ** combo_exp)
    bubble_colors = mono_cmap(norm_imp)

    # 绘制气泡 (无变化)
    for i in range(len(df)):
        ax.scatter(
            df["均值_SHAP"][i], df["SHAP_标准差"][i],
            s=size[i], color=bubble_colors[i],
            alpha=0.77, linewidths=0, zorder=2
        )

    # 添加标签 (字体已是Times New Roman, bold，无需修改)
    texts = []
    for i in range(len(df)):
        if size[i] > 2200:
            texts.append(ax.text(
                df["均值_SHAP"][i], df["SHAP_标准差"][i],
                str(df["变量"][i]),
                fontsize=24 + int(15 * combo[i]),
                color='black',
                ha='center', va='center', zorder=10,
                fontweight='bold', fontfamily='Times New Roman',
                bbox=dict(boxstyle="round,pad=0.18", fc="white", alpha=0.82, lw=0)
            ))
        else:
            dx = 0.02 * (df["均值_SHAP"].max() - df["均值_SHAP"].min())
            dy = 0.02 * (df["SHAP_标准差"].max() - df["SHAP_标准差"].min())
            texts.append(ax.text(
                df["均值_SHAP"][i] + dx, df["SHAP_标准差"][i] + dy,
                str(df["变量"][i]),
                fontsize=30 + int(6 * combo[i]),
                color='black',
                ha='left', va='bottom', zorder=10,
                fontweight='bold', fontfamily='Times New Roman',
                bbox=dict(boxstyle="round,pad=0.13", fc="white", alpha=0.75, lw=0)
            ))

    smart_adjust_text(texts, ax)

    # 坐标轴 & 样式 (轴标签已是Times New Roman, bold，无需修改)
    ax.set_ylabel("Morris σ", fontsize=40, fontweight='bold',
                  color='black', labelpad=15, fontfamily='Times New Roman')
    ax.set_xlabel("Morris µ*", fontsize=50, fontweight='bold',
                  color='black', fontfamily='Times New Roman', labelpad=11)
    ax.grid(True, linestyle="--", alpha=0.13, zorder=0)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.tick_params(axis='y', labelsize=30)
    ax.tick_params(axis='x', labelsize=30)

    # --- 修改：将坐标轴刻度字体设置为 Times New Roman 并加粗 ---
    for label in ax.get_yticklabels():
        label.set_fontname('Times New Roman')
        label.set_fontweight('bold') # 加粗
    for label in ax.get_xticklabels():
        label.set_fontname('Times New Roman')
        label.set_fontweight('bold') # 加粗

# 红色系 colormap (无变化)
mono_cmap = plt.cm.Reds

# 1x2 布局 (无变化)
fig, axes = plt.subplots(1, 2, figsize=(20, 20), sharex=False, constrained_layout=False)
axes = axes.flatten()

for i, (csv_path, label) in enumerate(csv_files):
    df = pd.read_csv(csv_path, encoding="utf-8-sig")
    ax = axes[i]
    plot_morris(df, ax, mono_cmap)

    # 子图角标 (字体已是Times New Roman, bold，无需修改)
    ax.text(0.01, 0.97, label, transform=ax.transAxes,
            fontsize=45, fontweight='bold', va='top', ha='left',
            color='black', fontfamily='Times New Roman')

# 色条 (无变化)
all_min = min(pd.read_csv(f[0], encoding="utf-8-sig")["特征重要性"].min() for f in csv_files)
all_max = max(pd.read_csv(f[0], encoding="utf-8-sig")["特征重要性"].max() for f in csv_files)
norm = plt.Normalize(vmin=all_min, vmax=all_max)
sm = plt.cm.ScalarMappable(cmap=mono_cmap, norm=norm)
sm.set_array([])

# 色条参数 (无变化)
cbar = fig.colorbar(
    sm,
    ax=axes.ravel().tolist(),
    orientation='vertical',
    shrink=0.75,
    pad=0.03,
    fraction=0.05,
    aspect=20
)
# --- 修改：移除色条标签文本 ---
cbar.set_label(
    "", # 将 "特征重要性" 变为空字符串
    fontsize=45, fontweight='bold', fontfamily='Times New Roman', color='black', labelpad=28
)

cbar.ax.tick_params(colors='black', labelsize=25)
# 色条刻度字体设置 (已是Times New Roman, bold，无需修改)
for label in cbar.ax.get_yticklabels():
    label.set_fontname('Times New Roman')
    label.set_fontweight('bold')

for spine in cbar.ax.spines.values():
    spine.set_edgecolor('black')

# 整体布局参数 (无变化)
plt.subplots_adjust(left=0.07, right=0.9, top=0.95, bottom=0.12,
                      hspace=0, wspace=0.28)
plt.savefig(outpath, dpi=340, transparent=True)
plt.close()

print(f"[✔] 图像已成功保存至：{outpath}")

[✔] 图像已成功保存至：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\图像优化\去除PEO_剩FEI_SEEO\XGBoost_Morris优化_去PEO.png


# SHAP依赖图

## 全部三个目标变量

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import os
from xgboost import XGBRegressor
from matplotlib.colors import Normalize

matplotlib.rcParams['font.sans-serif'] = ['Times New Roman']
matplotlib.rcParams['axes.unicode_minus'] = False

columns = [
    {
        "title": "FEI", "cmap": plt.cm.Reds, "plots": [
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\fei\2_确认非线性与变量交互\fei_SHAP_dependence_eced.csv", "x": "eced", "y": "SHAP_eced", "color": "seen", "xlabel": "eced", "inter_var": "seen"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\fei\2_确认非线性与变量交互\fei_SHAP_dependence_pe.csv", "x": "pe", "y": "SHAP_pe", "color": "fet", "xlabel": "pe", "inter_var": "fet"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\fei\2_确认非线性与变量交互\fei_SHAP_dependence_seeh.csv", "x": "seeh", "y": "SHAP_seeh", "color": "eced", "xlabel": "seeh", "inter_var": "eced"},
        ]
    },
    {
        "title": "PEO", "cmap": plt.cm.Greens, "plots": [
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\peo\2_确认非线性与变量交互\peo_SHAP_dependence_fel.csv", "x": "fel", "y": "SHAP_fel", "color": "pe", "xlabel": "fel", "inter_var": "pe"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\peo\2_确认非线性与变量交互\peo_SHAP_dependence_pe.csv", "x": "pe", "y": "SHAP_pe", "color": "fel", "xlabel": "pe", "inter_var": "fel"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\peo\2_确认非线性与变量交互\peo_SHAP_dependence_seeo.csv", "x": "seeo", "y": "SHAP_seeo", "color": "pe", "xlabel": "seeo", "inter_var": "pe"},
        ]
    },
    {
        "title": "SEEO", "cmap": plt.cm.Purples, "plots": [
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\seeo\2_确认非线性与变量交互\seeo_SHAP_dependence_csc.csv", "x": "csc", "y": "SHAP_csc", "color": "seegwoc", "xlabel": "csc", "inter_var": "seegwoc"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\seeo\2_确认非线性与变量交互\seeo_SHAP_dependence_peowc.csv", "x": "peowc", "y": "SHAP_peowc", "color": "csc", "xlabel": "peowc", "inter_var": "csc"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\seeo\2_确认非线性与变量交互\seeo_SHAP_dependence_seebwoc.csv", "x": "seebwoc", "y": "SHAP_seebwoc", "color": "cscf", "xlabel": "seebwoc", "inter_var": "cscf"},
        ]
    }
]

outdir = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\图像优化"
os.makedirs(outdir, exist_ok=True)
outpath = os.path.join(outdir, "XGBoost_SHAP依赖3x3矩阵_按列字母.png")

letter_labels = [
    ['a', 'b', 'c'],
    ['d', 'e', 'f'],
    ['g', 'h', 'i'],
]

fig, axes = plt.subplots(3, 3, figsize=(18, 15), constrained_layout=True)

for col, col_info in enumerate(columns):
    for row, plot in enumerate(col_info["plots"]):
        df = pd.read_csv(plot['csv'], encoding="utf-8-sig")
        x = df[plot['x']].values.reshape(-1, 1)
        y = df[plot['y']].values
        color_val = df[plot['color']].values

        if np.nanmax(color_val) == np.nanmin(color_val):
            seen_norm = np.full(len(color_val), 0.5)
        else:
            seen_norm = (color_val - np.nanmin(color_val)) / (np.nanmax(color_val) - np.nanmin(color_val) + 1e-9)
            seen_norm = np.clip(seen_norm, 0, 1)
        adjusted_seen_norm = 0.25 + 0.75 * seen_norm

        colors = col_info['cmap'](adjusted_seen_norm)
        size = 35 + (1 - seen_norm) * 80
        size = np.where(np.isnan(size), 35, size)
        size[size <= 0] = 10

        ax = axes[row, col]
        sc = ax.scatter(
            x, y, c=colors, s=size, alpha=0.88
        )

        xfit = np.linspace(np.nanmin(x), np.nanmax(x), 300).reshape(-1, 1)
        xgb = XGBRegressor(
            n_estimators=80, max_depth=3, learning_rate=0.07,
            random_state=2024, subsample=0.9
        )
        xgb.fit(x, y)
        yfit = xgb.predict(xfit)
        ax.plot(xfit, yfit, linestyle='--', color=col_info['cmap'](0.88), linewidth=2)

        # 修改横纵坐标字体
        ax.set_xlabel(plot['xlabel'], fontsize=30, fontweight='bold', fontname='Times New Roman')
        if col == 0:
            ax.set_ylabel("SHAP", fontsize=30, fontweight='bold', fontname='Times New Roman')
        else:
            ax.set_ylabel("")

        # 刻度字体
        ax.tick_params(axis='both', labelsize=20)
        for label in ax.get_xticklabels() + ax.get_yticklabels():
            label.set_fontname('Times New Roman')
            label.set_fontweight('bold')

        ax.grid(True, linestyle='--', alpha=0.22, zorder=0)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        # 字母标注
        ax.text(0.01, 0.97, letter_labels[col][row], transform=ax.transAxes,
                fontsize=22, fontweight='bold', va='top', ha='left', color=col_info['cmap'](0.82),
                fontname='Times New Roman')

        # 色条
        norm = Normalize(vmin=0.25, vmax=1)
        cbar = plt.colorbar(
            plt.cm.ScalarMappable(cmap=col_info['cmap'], norm=norm),
            ax=ax, pad=0.012, shrink=0.9
        )
        cbar.set_label(plot['inter_var'], fontsize=30, fontweight='bold', fontname='Times New Roman')
        cbar.ax.tick_params(labelsize=15)
        cbar.ax.set_yticklabels([])

# 顶部主标题
for col, col_info in enumerate(columns):
    axes[0, col].set_title(
        f"{col_info['title']}",
        fontsize=30, fontweight='bold', color='black', pad=18, loc='left', fontname='Times New Roman'
    )

plt.savefig(outpath, dpi=360)
plt.close()
print(f"[✔] 3×3矩阵（列式字母a-i）已保存至：{outpath}")

[✔] 3×3矩阵（列式字母a-i）已保存至：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\图像优化\XGBoost_SHAP依赖3x3矩阵_按列字母.png


## 去掉PEO

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import os
from xgboost import XGBRegressor
from matplotlib.colors import Normalize

matplotlib.rcParams['font.sans-serif'] = ['Times New Roman']
matplotlib.rcParams['axes.unicode_minus'] = False

columns = [
    {
        "title": "FEI", "cmap": plt.cm.Reds, "plots": [
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\fei\2_确认非线性与变量交互\fei_SHAP_dependence_eced.csv", "x": "eced", "y": "SHAP_eced", "color": "seen", "xlabel": "eced", "inter_var": "seen"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\fei\2_确认非线性与变量交互\fei_SHAP_dependence_pe.csv", "x": "pe", "y": "SHAP_pe", "color": "fet", "xlabel": "pe", "inter_var": "fet"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\fei\2_确认非线性与变量交互\fei_SHAP_dependence_seeh.csv", "x": "seeh", "y": "SHAP_seeh", "color": "eced", "xlabel": "seeh", "inter_var": "eced"},
        ]
    },
    {
        "title": "SEEO", "cmap": plt.cm.Purples, "plots": [
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\seeo\2_确认非线性与变量交互\seeo_SHAP_dependence_csc.csv", "x": "csc", "y": "SHAP_csc", "color": "seegwoc", "xlabel": "csc", "inter_var": "seegwoc"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\seeo\2_确认非线性与变量交互\seeo_SHAP_dependence_peowc.csv", "x": "peowc", "y": "SHAP_peowc", "color": "csc", "xlabel": "peowc", "inter_var": "csc"},
            {"csv": r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\seeo\2_确认非线性与变量交互\seeo_SHAP_dependence_seebwoc.csv", "x": "seebwoc", "y": "SHAP_seebwoc", "color": "cscf", "xlabel": "seebwoc", "inter_var": "cscf"},
        ]
    }
]

# --- 修改点: 更新输出文件夹和文件名 ---
outdir = r"C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\图像优化\去除PEO_剩FEI_SEEO"
os.makedirs(outdir, exist_ok=True)
outpath = os.path.join(outdir, "XGBoost_SHAP依赖_去PEO.png")

letter_labels = [
    ['a', 'b', 'c'],  # 第一列 (FEI)
    ['d', 'e', 'f'],  # 第二列 (SEEO)
]

fig, axes = plt.subplots(3, 2, figsize=(12, 15), constrained_layout=True)

for col, col_info in enumerate(columns):
    for row, plot in enumerate(col_info["plots"]):
        df = pd.read_csv(plot['csv'], encoding="utf-8-sig")
        x = df[plot['x']].values.reshape(-1, 1)
        y = df[plot['y']].values
        color_val = df[plot['color']].values

        if np.nanmax(color_val) == np.nanmin(color_val):
            seen_norm = np.full(len(color_val), 0.5)
        else:
            seen_norm = (color_val - np.nanmin(color_val)) / (np.nanmax(color_val) - np.nanmin(color_val) + 1e-9)
            seen_norm = np.clip(seen_norm, 0, 1)
        adjusted_seen_norm = 0.25 + 0.75 * seen_norm

        colors = col_info['cmap'](adjusted_seen_norm)
        size = 35 + (1 - seen_norm) * 80
        size = np.where(np.isnan(size), 35, size)
        size[size <= 0] = 10

        ax = axes[row, col]
        sc = ax.scatter(
            x, y, c=colors, s=size, alpha=0.88
        )

        xfit = np.linspace(np.nanmin(x), np.nanmax(x), 300).reshape(-1, 1)
        xgb = XGBRegressor(
            n_estimators=80, max_depth=3, learning_rate=0.07,
            random_state=2024, subsample=0.9
        )
        xgb.fit(x, y)
        yfit = xgb.predict(xfit)
        ax.plot(xfit, yfit, linestyle='--', color=col_info['cmap'](0.88), linewidth=2)

        ax.set_xlabel(plot['xlabel'], fontsize=30, fontweight='bold', fontname='Times New Roman')
        if col == 0:
            ax.set_ylabel("SHAP", fontsize=30, fontweight='bold', fontname='Times New Roman')
        else:
            ax.set_ylabel("")

        ax.tick_params(axis='both', labelsize=20)
        for label in ax.get_xticklabels() + ax.get_yticklabels():
            label.set_fontname('Times New Roman')
            label.set_fontweight('bold')

        ax.grid(True, linestyle='--', alpha=0.22, zorder=0)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        ax.text(0.01, 0.97, letter_labels[col][row], transform=ax.transAxes,
                fontsize=22, fontweight='bold', va='top', ha='left', color=col_info['cmap'](0.82),
                fontname='Times New Roman')

        norm = Normalize(vmin=0.25, vmax=1)
        cbar = plt.colorbar(
            plt.cm.ScalarMappable(cmap=col_info['cmap'], norm=norm),
            ax=ax, pad=0.012, shrink=0.9
        )
        cbar.set_label(plot['inter_var'], fontsize=30, fontweight='bold', fontname='Times New Roman')
        cbar.ax.tick_params(labelsize=15)
        cbar.ax.set_yticklabels([])

# 顶部主标题
for col, col_info in enumerate(columns):
    axes[0, col].set_title(
        f"{col_info['title']}",
        fontsize=30, fontweight='bold', color='black', pad=18, loc='left', fontname='Times New Roman'
    )

plt.savefig(outpath, dpi=360)
plt.close()
print(f"[✔] 图像已保存至：{outpath}")

[✔] 图像已保存至：C:\Users\phc\Desktop\中国模型比较\中国模型比较2\4_机器学习归因\XGBoost\var_attri\results\图像优化\去除PEO_剩FEI_SEEO\XGBoost_SHAP依赖_去PEO.png
